# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [95]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
School_data = pd.DataFrame(school_data)
student_data = pd.read_csv(student_data_to_load)
Student_data = pd.DataFrame(student_data)

# Combine the data into a single dataset (consider using a left join)
merge_df = pd.merge(Student_data, School_data, on="school_name", how="left")
merge_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0.0,District,2917.0,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0.0,District,2917.0,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0.0,District,2917.0,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0.0,District,2917.0,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0.0,District,2917.0,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14.0,Charter,1635.0,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14.0,Charter,1635.0,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14.0,Charter,1635.0,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14.0,Charter,1635.0,1043130


## District Summary

In [96]:
# Calculate the Totals (Schools and Students)
Schools = len(merge_df.school_name.unique())
Schools
Students = len(merge_df.student_name)
Students
# Calculate the Total Budget
Budget = sum(School_data.budget)
Budget
# Calculate the Average Scores
Math_Average = merge_df.math_score.mean()
Math_Average
Reading_Average = merge_df.reading_score.mean()
Reading_Average
# Calculate the Percentage Pass Rates
math_pass = merge_df.loc[merge_df["math_score"] > 69]
math_pass.reset_index(inplace = True)
Math_pass = round((len(math_pass.math_score)/Students)*100, 6)
Math_pass
reading_pass = merge_df.loc[merge_df["reading_score"] > 69]
reading_pass.reset_index(inplace=True)
reading_pass
Reading_pass = round((len(reading_pass.reading_score)/Students)*100, 6)
Reading_pass
Total_pass = (Math_pass + Reading_pass) / 2
Total_pass
# Minor Data Cleanup

# Display the data frame
Summary_df = pd.DataFrame({'Total Schools':[Schools],
                          'Total Students':[Students],
                          'Total Budget':[Budget],
                          'Average Math Score':[Math_Average],
                          'Average Reading Score':[Reading_Average],
                          '% Passing Math':[Math_pass],
                          '% Passing Reading':[Reading_pass],
                          '% Overall Passing Rate':[Total_pass]})
Summary_df["Total Budget"] = Summary_df["Total Budget"].map("${:,.2f}".format)
Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$49,298,856.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

In [97]:
# Determine the School Type
School_type = pd.DataFrame(School_data[['school_name', 'type']])
School_type.sort_values('school_name', inplace=True)
School_type.reset_index(drop=True)
# Calculate the total student count
student_count = Student_data.groupby(['school_name'])
Student_Count = student_count['student_name']
Student_Count= pd.DataFrame(Student_Count.count())
Student_Count.reset_index(inplace=True)
Student_Count = Student_Count.rename(columns={"student_name":"Student Total"})
# Merge the School Type and Student Type into a New Chart
school_merge = pd.merge(School_type, Student_Count, on="school_name", how="outer")
school_merge = school_merge.rename(columns={"type":"District Type"})
# Calculate the total school budget and per capita spending
per_school_budget = School_data.groupby(["school_name"]).mean()
per_school_budget['Per Student_Budget'] = per_school_budget['budget']/per_school_budget['size']
Per_School_Budget= pd.DataFrame(per_school_budget[['budget','Per Student_Budget']])
#Merge again
Merge_df = pd.merge(school_merge, Per_School_Budget, on = "school_name", how="outer")
Merge_df = Merge_df.rename(columns={'budget':"Total School Budget", "Per Student_Budget":"Per Student Budget"})
# Calculate the average test scores
Average_Test = merge_df.groupby(["school_name"]).mean()
Average_Merge = Average_Test[['reading_score','math_score']]
Average_Merge = Average_Merge.rename(columns={"reading_score":"Average Reading Score", "math_score":"Average Math Score"})
Average_Merge.reset_index(inplace=True)
#Merge Average Merge with Merge_df
Next_Merge = pd.merge(Merge_df, Average_Merge, on= "school_name", how="outer")
# Calculate the passing scores by creating a filtered data frame
# Math Passing Chart
Math_Pass = Student_data.loc[Student_data["math_score"] > 69]
Math_Pass_By_School = Math_Pass.groupby("school_name").count()
Math_Percentage = pd.merge(Student_Count, Math_Pass_By_School, on="school_name", how="outer")
Math_Percentage['% Passing Math'] = round((Math_Percentage['math_score'] / Math_Percentage['Student Total'])*100, 2)
Math_Percentage.reset_index(inplace = True)
Math_Percentage = Math_Percentage[["school_name", "% Passing Math"]]
Math_Merge = pd.merge(Next_Merge, Math_Percentage, on="school_name", how="outer")
# Reading Passing Chart
Reading_Pass = Student_data.loc[Student_data["reading_score"] > 69]
Reading_Pass_By_School = Reading_Pass.groupby("school_name").count()
Reading_Percentage = pd.merge(Student_Count, Reading_Pass_By_School, on="school_name", how="outer")
Reading_Percentage['% Passing Reading'] = round((Reading_Percentage['reading_score'] / Reading_Percentage['Student Total'])*100, 2)
Reading_Percentage.reset_index(inplace = True)
Reading_Percentage = Reading_Percentage[["school_name", "% Passing Reading"]]
Reading_Merge = pd.merge(Math_Merge, Reading_Percentage, on="school_name", how="outer")
# Overall Passing Chart
Reading_Merge['% Overall Passing Rate'] = round((Reading_Merge['% Passing Math'] + Reading_Merge['% Passing Reading'])/2, 2)
Reading_Merge = Reading_Merge.rename(columns={"school_name":"School Name"})
Reading_Merge["Total School Budget"] = Reading_Merge["Total School Budget"].map("${:,.2f}".format)
Reading_Merge

,School Name,District Type,Student Total,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976.0,"$3,124,928.00",628.0,81.033963,77.048432,66.68,81.93,74.31
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",582.0,83.975780,83.061895,94.13,97.04,95.58
2,Figueroa High School,District,2949.0,"$1,884,411.00",639.0,81.158020,76.711767,65.99,80.74,73.36
3,Ford High School,District,2739.0,"$1,763,916.00",644.0,80.746258,77.102592,68.31,79.30,73.81
4,Griffin High School,Charter,1468.0,"$917,500.00",625.0,83.816757,83.351499,93.39,97.14,95.26
5,Hernandez High School,District,4635.0,"$3,022,020.00",652.0,80.934412,77.289752,66.75,80.86,73.81
6,Holden High School,Charter,427.0,"$248,087.00",581.0,83.814988,83.803279,92.51,96.25,94.38
7,Huang High School,District,2917.0,"$1,910,635.00",655.0,81.182722,76.629414,65.68,81.32,73.50
8,Johnson High School,District,4761.0,"$3,094,650.00",650.0,80.966394,77.072464,66.06,81.22,73.64
9,Pena High School,Charter,962.0,"$585,858.00",609.0,84.044699,83.839917,94.59,95.95,95.27


In [52]:
Reading_Merge["Per Student Budget"] = Reading_Merge["Per Student Budget"].map("${:,.2f}".format)
Reading_Merge

,School Name,District Type,Student Total,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976.0,"$3,124,928.00",$628.00,81.033963,77.048432,66.68,81.93,74.31
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",$582.00,83.975780,83.061895,94.13,97.04,95.58
2,Figueroa High School,District,2949.0,"$1,884,411.00",$639.00,81.158020,76.711767,65.99,80.74,73.36
3,Ford High School,District,2739.0,"$1,763,916.00",$644.00,80.746258,77.102592,68.31,79.30,73.81
4,Griffin High School,Charter,1468.0,"$917,500.00",$625.00,83.816757,83.351499,93.39,97.14,95.26
5,Hernandez High School,District,4635.0,"$3,022,020.00",$652.00,80.934412,77.289752,66.75,80.86,73.81
6,Holden High School,Charter,427.0,"$248,087.00",$581.00,83.814988,83.803279,92.51,96.25,94.38
7,Huang High School,District,2917.0,"$1,910,635.00",$655.00,81.182722,76.629414,65.68,81.32,73.50
8,Johnson High School,District,4761.0,"$3,094,650.00",$650.00,80.966394,77.072464,66.06,81.22,73.64
9,Pena High School,Charter,962.0,"$585,858.00",$609.00,84.044699,83.839917,94.59,95.95,95.27


## Top Performing Schools (By Passing Rate)

In [98]:
# Sort and show top five schools
Reading_Merge.sort_values('% Overall Passing Rate', inplace=True, ascending = False)
Reading_Merge.head(5)

,School Name,District Type,Student Total,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
1,Cabrera High School,Charter,1858.0,"$1,081,356.00",582.0,83.975780,83.061895,94.13,97.04,95.58
12,Thomas High School,Charter,1635.0,"$1,043,130.00",638.0,83.848930,83.418349,93.27,97.31,95.29
9,Pena High School,Charter,962.0,"$585,858.00",609.0,84.044699,83.839917,94.59,95.95,95.27
4,Griffin High School,Charter,1468.0,"$917,500.00",625.0,83.816757,83.351499,93.39,97.14,95.26
13,Wilson High School,Charter,2283.0,"$1,319,574.00",578.0,83.989488,83.274201,93.87,96.54,95.21


## Bottom Performing Schools (By Passing Rate)

In [99]:
# Sort and show bottom five schools
Reading_Merge.sort_values('% Overall Passing Rate', inplace=True)
Reading_Merge.head(5)

,School Name,District Type,Student Total,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
10,Rodriguez High School,District,3999.0,"$2,547,363.00",637.0,80.744686,76.842711,66.37,80.22,73.30
2,Figueroa High School,District,2949.0,"$1,884,411.00",639.0,81.158020,76.711767,65.99,80.74,73.36
7,Huang High School,District,2917.0,"$1,910,635.00",655.0,81.182722,76.629414,65.68,81.32,73.50
8,Johnson High School,District,4761.0,"$3,094,650.00",650.0,80.966394,77.072464,66.06,81.22,73.64
3,Ford High School,District,2739.0,"$1,763,916.00",644.0,80.746258,77.102592,68.31,79.30,73.81


## Math Scores by Grade

In [100]:
# Create data series of scores by grade levels using conditionals
ninth_grade = Student_data[(Student_data['grade'] == "9th")]
tenth_grade = Student_data[(Student_data['grade'] == '10th')]
eleventh_grade = Student_data[(Student_data['grade'] == '11th')]
twelvth_grade = Student_data[(Student_data['grade'] == '12th')]
# Group by School_Name
Ninth_grade = ninth_grade.groupby(['school_name']).mean()['math_score']
Tenth_grade = tenth_grade.groupby(['school_name']).mean()['math_score']
Eleventh_grade = eleventh_grade.groupby(['school_name']).mean()['math_score']
Twelvth_grade = twelvth_grade.groupby(['school_name']).mean()['math_score']
# Combine Data Series 
Grade_Scores = pd.DataFrame({"9th": Ninth_grade, "10th":Tenth_grade, "11th":Eleventh_grade,"12th":Twelvth_grade})
# Minor data munging
Grade_Scores = Grade_Scores[["9th", '10th', '11th', '12th']]
Grade_Scores.index.name = None
# Display the data frame
Grade_Scores

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [101]:
# Create data series of scores by grade levels using conditionals
ninth_grade = Student_data[(Student_data['grade'] == "9th")]
tenth_grade = Student_data[(Student_data['grade'] == '10th')]
eleventh_grade = Student_data[(Student_data['grade'] == '11th')]
twelvth_grade = Student_data[(Student_data['grade'] == '12th')]
# Group each by school name
Ninth_grade = ninth_grade.groupby(['school_name']).mean()['reading_score']
Tenth_grade = tenth_grade.groupby(['school_name']).mean()['reading_score']
Eleventh_grade = eleventh_grade.groupby(['school_name']).mean()['reading_score']
Twelvth_grade = twelvth_grade.groupby(['school_name']).mean()['reading_score']
# Combine series into single data frame
Grade_Scores = pd.DataFrame({"9th": Ninth_grade, "10th":Tenth_grade, "11th":Eleventh_grade,"12th":Twelvth_grade})
# Minor data munging
Grade_Scores = Grade_Scores[["9th", '10th', '11th', '12th']]
Grade_Scores.index.name = None
# Display the data frame
Grade_Scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [106]:
# Establish the bins -- choose any set of bins you would like, but see below for testing bins
# to test, set your bins as follows: [0, 585, 615, 645, 675]
# ALSO -- Note that the values for `% Passing Math`, `% Passing Reading` and `% Overall Passing Rate`
# were computed using averages of averages -- your results may vary if you use weighted averages 
bins = [0, 585, 615, 645, 675]
bin_name = ['<$585', '$585-615','$615-645', '$645-675']
Reading_Merge["Spending Ranges (Per Student)"] = pd.cut(Reading_Merge["Per Student Budget"], bins, labels=bin_name, include_lowest=True)
# Categorize the spending based on the bins
Reading_Merge_Summary = pd.DataFrame(Reading_Merge.groupby(['Spending Ranges (Per Student)']).mean())
# Round Several Columns
#Reading_Merge_Summary['Per Student Budget'] = round(Reading_Merge_Summary['Average Reading Score'], 2)
Reading_Merge_Summary['Average Reading Score'] = round(Reading_Merge_Summary['Average Reading Score'], 2)
Reading_Merge_Summary['Average Math Score'] = round(Reading_Merge_Summary['Average Math Score'], 2)
Reading_Merge_Summary['% Passing Math'] = round(Reading_Merge_Summary['% Passing Math'], 2)
Reading_Merge_Summary['% Passing Reading'] = round(Reading_Merge_Summary['% Passing Reading'], 2)
Reading_Merge_Summary['% Overall Passing Rate'] = round(Reading_Merge_Summary['% Overall Passing Rate'], 2)
# Minor data munging
#Reading_Merge_Summary["Per Student Budget"] = Reading_Merge_Summary["Per Student Budget"].map("${:,.2f}".format)
Reading_Merge_Summary = Reading_Merge_Summary[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                             '% Passing Reading', '% Overall Passing Rate']]
# Display results
Reading_Merge_Summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.06
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


## Scores by School Size

In [103]:
# Establish the bins 
bins = [0, 1000, 2000, 5000]
# Categorize the spending based on the bins
bin_name = ['Small(<1000)', 'Medium(1000-2000)','Large(2000-5000)']
# Calculate the scores based on bins
Reading_Merge["School Size"] = pd.cut(Reading_Merge["Student Total"], bins, labels=bin_name, include_lowest=True)
Reading_Merge
# Assemble into data frame
School_Size = Reading_Merge.groupby("School Size").mean()
School_Size = School_Size[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# Display results
School_Size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,93.55000,96.10000,94.8250
Medium(1000-2000),83.374684,83.864438,93.59800,96.79000,95.1920
Large(2000-5000),77.746417,81.344493,69.96375,82.76625,76.3675


## Scores by School Type

In [104]:
# Assemble into data frame
Score = Reading_Merge.groupby("District Type").mean()
Score = Score[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# Display results
Score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
District Type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.102500
District,76.956733,80.966636,66.548571,80.798571,73.675714
